<br><br>

<div style='text-align: center; font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;'>
  <span style='font-size: 30px;'>📊</span>
  <span style='font-weight: bold; font-size: 32px; color: #00AF6C;'>Notebook que cria a tabela silver</span>
</div>

<br>

<div style='text-align: center; font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;'>
  <span style='font-weight: normal; font-size: 19px; color: #F;'>Esse notebook cria a versão silver da tabela em data</span>
</div>

In [1]:
import pandas as pd
import numpy as np
import os, sys
from datetime import datetime

sys.path.insert(0, os.path.abspath(".."))
from src import utils as ult
from dotenv import load_dotenv ; load_dotenv()

# Desligando warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Coleta o path_start, basicamente o local da pasta onde estão os dados.
path_start = os.getenv("PATH_START")
df_first = ult.load_data(path=f"{path_start}data\\dados_stone.xlsx", sheet_name='Aba 1')
df_second = ult.load_data(path=f"{path_start}data\\dados_stone.xlsx", sheet_name='Aba 2')

### Arrumando a replicação incorreta dos dados nas últimas 124 linhas

In [3]:
df_fixing = df_first.query("CBK != CBK")
df_fixing['Dia_test'] = list(
    map(
        lambda dia: dia.replace(hour=0, minute=0, second=0, microsecond=0)
        ,df_fixing['Dia']
    )
)

df_fixing['Hora_test'] = list(
    map(
        lambda dia: dia.time()
        ,df_fixing['Dia']
    )
)

df_fixing['Valor_test'] = df_fixing['Hora']
df_fixing['Cartão_test'] = df_fixing['Valor']
df_fixing['CBK_test'] = df_fixing['Cartão']

In [4]:
# Padronizando os nomes das colunas, selecionando apenas as _test
df_fixing = df_fixing.filter(
    [col for col in df_fixing.columns.to_list() if col.endswith('_test')]
)

# Retirando o _test das colunas
df_fixing.columns = [col[:-5] for col in df_fixing.columns.to_list()]

# Realizando o Join final
df_final = pd.concat([df_first.iloc[:df_fixing.index.min()], df_fixing], axis=0)

### Juntando ambos os dataframes

In [6]:
df_final['label'] = 'trainer'
df_second['label'] = 'payload'

df_final = pd.concat([df_final, df_second],axis=0)

### Escrevendo os dados

In [8]:
df_final.to_csv(f"{path_start}data\\silver_dados_stone.csv")